In [91]:
import copy


class Item:
    def __init__(self, item: str, utility: int):
        self.item = item
        self.utility = utility
        self._twu = 0

    @property
    def twu(self) -> int:
        return self._twu

    @twu.setter
    def twu(self, value: int) -> None:
        self._twu = value

    def __repr__(self):
        # return f"({self.item},{self.twu})"
        return f"{self.item}"

    def __eq__(self, other):
        if isinstance(other, Item):
            return self.item == other.item and self.utility == other.utility
        return False

    def __hash__(self):
        return hash((self.item, self.utility))


class TransItem:
    def __init__(self, item: Item, quantity: int, probability: float):
        self.item = item
        self.quantity = quantity
        self.probability = probability

    def __repr__(self):
        return f"{self.item},{self.quantity},{self.probability})"

    def get_total_probability(self):
        return self.quantity * self.probability


def check_order_condition(a: Item, b: Item):
    if a.utility < 0 and b.utility > 0:
        return True
    elif a.utility * b.utility > 0:
        return a.twu > b.twu
    return False


def check_order_item_and_set(ik: Item, X: set[Item]) -> bool:
    for i in X:
        if i != ik and check_order_condition(ik, i) == False:
            return False
    return True


class Transaction:
    def __init__(self, id: int, trans_items: set[TransItem]):
        self.id = id
        self.trans_items = trans_items

    def __repr__(self):
        return f"t{self.id}, {self.trans_items}"

    def contains_item_set(self, item_set: set[Item]) -> bool:
        transaction_items = {trans_item.item for trans_item in self.trans_items}
        return item_set.issubset(transaction_items)

    def get_quantity_of_item(self, item: Item) -> int:
        for trans_item in self.trans_items:
            if trans_item.item == item:
                return trans_item.quantity
        return 0

    def get_probability_of_item(self, item: Item) -> int:
        for trans_item in self.trans_items:
            if trans_item.item == item:
                return trans_item.probability
        return 0

    def get_items(self) -> set[Item]:
        return {trans_item.item for trans_item in self.trans_items}

    def get_probability_of_item_set(self, item_set: set[Item]) -> float:
        total_probability = 0.0
        if self.contains_item_set(item_set):
            total_probability = 1.0
            for trans_item in self.trans_items:
                if trans_item.item in item_set:
                    total_probability *= trans_item.probability
        return total_probability

    def get_positive_utility_of_item_set(self, items: set[Item]):
        pu = 0
        for item in items:
            utility = item.utility
            if utility > 0:
                quantity = self.get_quantity_of_item(item)
                pu += item.utility * quantity
        return pu

    def get_negative_utility_of_item_set(self, items: set[Item]):
        nu = 0
        for item in items:
            utility = item.utility
            if utility < 0:
                quantity = self.get_quantity_of_item(item)
                nu += item.utility * quantity
        return nu

    def get_remaining_utility_of_item_set(self, items: set[Item]):
        ru = 0
        trans_items: set[Item] = self.get_items()
        for item in trans_items:
            if item.utility > 0:
                if item not in items:
                    if check_order_item_and_set(item, items) == True:
                        ru += item.utility * self.get_quantity_of_item(item)
        return ru


from collections import namedtuple

Utilities = namedtuple("Utilities", ["tid", "pro", "pu", "nu", "ru"])


class AbstractList:
    def __init__(self, items: set[Item], utility_values: list[Utilities]):
        self.items = items
        self.utility_values = utility_values

    def get_ru(self):
        ru = 0
        for i in self.utility_values:
            ru += i.ru
        return ru

    def get_pu(self):
        pu = 0
        for i in self.utility_values:
            pu += i.pu
        return pu

    def get_nu(self):
        nu = 0
        for i in self.utility_values:
            nu += i.nu
        return nu

    def get_pro(self):
        pro = 0
        for i in self.utility_values:
            pro += i.pro
        return pro

    def __repr__(self):
        if not self.utility_values:
            return "Empty PNU-List"

        # Column titles
        titles = ["PRO", "PU", "NU", "RU"]

        # Get the number of columns from the first utility value
        if isinstance(self.utility_values[0], (list, tuple)):
            num_columns = len(self.utility_values[0])
        else:
            num_columns = 1

        # Create combined items string
        items_str = ",".join(str(item) for item in self.items)
        items_str = "(" + items_str + ")"

        # Calculate column widths based on utility values and titles
        value_widths = []
        for i in range(num_columns):
            max_width = max(
                len(str(row[i])) if isinstance(row, (list, tuple)) else len(str(row))
                for row in self.utility_values
            )
            # Consider width of titles and combined items
            if i == 0:
                max_width = max(max_width, len(items_str))
            else:
                max_width = max(max_width, len(titles[i - 1]))
            value_widths.append(max_width)

        # Build the table string
        result = []

        # Add border
        total_width = sum(value_widths) + 3 * num_columns + 1
        result.append("-" * total_width)

        # Add single row with all items and titles
        row = "|"
        row += f" {items_str.rjust(value_widths[0])} |"
        for i in range(1, num_columns):
            row += f" {titles[i-1].center(value_widths[i])} |"
        result.append(row)

        # Add separator
        result.append("-" * total_width)

        # Add utility values
        for utility in self.utility_values:
            row = "|"
            if isinstance(utility, (list, tuple)):
                for i, value in enumerate(utility):
                    row += f" {str(value).rjust(value_widths[i])} |"
            else:
                row += f" {str(utility).rjust(value_widths[0])} |"
            result.append(row)

        # Add bottom border
        result.append("-" * total_width)
        return "\n".join(result)

In [92]:
def calculate_positive_utility_of_transaction(trans: Transaction):
    pu = 0
    items: set[Item] = trans.get_items()
    for item in items:
        if item.utility > 0:
            pu += trans.get_quantity_of_item(item) * item.utility
    return pu

In [93]:
def calculate_transaction_weight_utility(items: set[Item], database: list[Transaction]):
    twu = 0
    for trans in database:
        if trans.contains_item_set(items):
            twu += calculate_positive_utility_of_transaction(trans)
    return twu

In [94]:
def calculate_positive_utility_of_item_set_in_trans(
    items: set[Item], trans: Transaction
):
    pu = 0
    for item in items:
        utility = item.utility
        if utility > 0:
            quantity = trans.get_quantity_of_item(item)
            pu += item.utility * quantity
    return pu

In [95]:
def calculate_negative_utility_of_item_set_in_trans(
    items: set[Item], trans: Transaction
):
    nu = 0
    for item in items:
        utility = item.utility
        if utility < 0:
            quantity = trans.get_quantity_of_item(item)
            nu += item.utility * quantity
    return nu

In [96]:
def check_order_condition(a: Item, b: Item):
    if a.utility < 0 and b.utility > 0:
        return True
    elif a.utility * b.utility > 0:
        return a.twu > b.twu
    return False


def check_order_item_and_set(ik: Item, X: set[Item]) -> bool:
    for i in X:
        if i != ik and check_order_condition(ik, i) == False:
            return False
    return True

In [97]:
def calculate_remaining_utility_of_item_set_in_trans(
    items: set[Item], trans: Transaction
):
    ru = 0
    trans_items: set[Item] = trans.get_items()
    for item in trans_items:
        if item.utility > 0:
            if check_order_item_and_set(item, items) == True:
                if item not in items:
                    ru += item.utility * trans.get_quantity_of_item(item)
    return ru

In [98]:
def create_eucs_dict(
    arr: list[Item], database: list[Transaction]
) -> dict[frozenset[Item], int]:
    n = len(arr)
    eucs_dict = {}
    for i in range(n):
        for j in range(n):
            if i < j:
                item_pair = frozenset({arr[i], arr[j]})
                twu_value = calculate_transaction_weight_utility(item_pair, database)
                eucs_dict[item_pair] = twu_value
    return eucs_dict

In [99]:
def find_tuple_by_trans_id(P: AbstractList, target_trans_id: int) -> Utilities:
    utilities_list: list[Utilities] = P.utility_values
    for iTuple in utilities_list:
        if iTuple.tid == target_trans_id:
            return iTuple
    return None


def construct(P: AbstractList, Px: AbstractList, Py: AbstractList):
    # print("start construct function")
    # print(Px)
    # print(Py)
    x = Px.items
    y = Py.items
    xy = x | y
    utilities_list: list[Utilities] = list()
    Pxy = AbstractList(xy, utilities_list)
    utilities_list_of_px: list[Utilities] = Px.utility_values
    for xTuple in utilities_list_of_px:
        yTuple: Utilities = find_tuple_by_trans_id(Py, xTuple.tid)
        if yTuple is not None:
            if P.utility_values:
                pTuple: Utilities = find_tuple_by_trans_id(P, xTuple.tid)
                xyTuple: Utilities = Utilities(
                    xTuple.tid,
                    0,
                    xTuple.pu + yTuple.pu - pTuple.pu,
                    xTuple.nu + yTuple.nu - pTuple.nu,
                    yTuple.ru,
                )
                utilities_list.append(xyTuple)
            else:
                xyTuple: Utilities = Utilities(
                    xTuple.tid,
                    0,
                    xTuple.pu + yTuple.pu,
                    xTuple.nu + yTuple.nu,
                    yTuple.ru,
                )
                utilities_list.append(xyTuple)
    return Pxy

In [100]:
def calculate_utility_of_item_set_in_database(
    items: set[Item], database: list[Transaction]
):
    u = 0
    for trans in database:
        if trans.contains_item_set(items):
            for item in items:
                u += item.utility * trans.get_quantity_of_item(item)
    return u

In [101]:
def sort_items_by_twu_and_utility(items: list[Item]) -> list[Item]:
    def sort_key(item: Item) -> tuple:
        return (0 if item.utility > 0 else 1, item.twu)

    return sorted(items, key=sort_key)

In [102]:
def searching_procedure(
    PList: AbstractList,
    lists: list[AbstractList],
    minUtil,
    minPro,
    database: list[Transaction],
    eucs_dict: dict[frozenset[Item], int],
    output: list[set[Item]]
):
    for i in range(0, len(lists) - 1):
        XList: AbstractList = lists[i]
        # print("X: " + str(XList.items))
        utility = calculate_utility_of_item_set_in_database(set(XList.items), database)
        remaining_utility = XList.get_ru()
        if (
            utility >= minUtil
        ):
            output.append(XList.items)
        if (
            remaining_utility + utility >= minUtil
        ):
            new_lists: list[AbstractList] = list()
            for j in range(i + 1, len(lists)):
                YList: AbstractList = lists[j]
                if True:
                    x = XList.items.difference(PList.items)
                    y = YList.items.difference(PList.items)
                    key = frozenset(x | y)
                    twu_value = eucs_dict.get(key)
                    if twu_value >= minUtil:
                        ZList = construct(PList, XList, YList)
                        new_lists.append(ZList)
                        # print(
                        #     str(XList.items)
                        #     + " U "
                        #     + str(YList.items)
                        #     + " = "
                        #     + str(ZList.items)
                        # )
            # print()
            searching_procedure(
                XList,
                new_lists,
                minUtil,
                minPro,
                database,
                eucs_dict,
                output
            )

In [103]:
def preparation_procedure(
    db: list[Transaction], item_list: list[Item], minUtility, minPro
):
    removed_list: list[Item] = list()
    # Check TWU & Periodic condition
    for item in item_list:
        twu = calculate_transaction_weight_utility({item}, db)
        if twu < minUtility:
            removed_list.append(item)
        else:
            item.twu = twu

    # Remove unqualified item
    new_distinct_items = [item for item in item_list if item not in removed_list]
    new_distinct_items = sort_items_by_twu_and_utility(new_distinct_items)
    print("new list: " + str(new_distinct_items))
    newDb = copy.deepcopy(db)

    # Remove unqualified item from transaction
    for trans in newDb:
        for item in removed_list:
            trans.items_quantities.pop(item, None)

    # Create list[AbstractList], prepare for algorithm 2
    lists: list[AbstractList] = list()

    for item in new_distinct_items:
        utility_values_list: list[tuple] = list()
        pnu_list = AbstractList({item}, utility_values_list)
        for trans in newDb:
            if trans.contains_item_set({item}):
                pro = trans.get_probability_of_item_set({item})
                pu = trans.get_positive_utility_of_item_set({item})
                nu = trans.get_negative_utility_of_item_set({item})
                ru = trans.get_remaining_utility_of_item_set({item})
                utility_values: Utilities = Utilities(trans.id, pro, pu, nu, ru)
                utility_values_list.append(utility_values)
        # print(pnu_list)
        lists.append(pnu_list)

    # Create EUCS
    eucs_dict: dict[frozenset[Item], int] = create_eucs_dict(new_distinct_items, newDb)

    root = AbstractList({}, list())
    # Call algorithm 2
    output = []
    searching_procedure(root, lists, minUtility, minPro, newDb, eucs_dict, output)
    return output

In [104]:
a = Item("a", 6)
b = Item("b", 7)
c = Item("c", 1)
d = Item("d", -5)
e = Item("e", 3)

item_list = [a, b, c, d, e]

t1_trans_items = {TransItem(b, 3, 0.85), TransItem(c, 1, 1.0), TransItem(d, 2, 0.70)}

t2_trans_items = {
    TransItem(a, 1, 1.0),
    TransItem(b, 1, 0.60),
    TransItem(c, 3, 0.75),
    TransItem(e, 1, 0.40),
}

t3_trans_items = {
    TransItem(a, 1, 0.55),
    TransItem(b, 2, 0.60),
    TransItem(c, 4, 1.0),
    TransItem(d, 1, 0.90),
    TransItem(e, 5, 0.40),
}

t4_trans_items = {TransItem(b, 3, 0.90), TransItem(d, 1, 0.45)}

t5_trans_items = {
    TransItem(a, 4, 1.0),
    TransItem(c, 3, 0.85),
    TransItem(d, 2, 0.70),
    TransItem(e, 2, 0.45),
}

t1 = Transaction(1, t1_trans_items)
t2 = Transaction(2, t2_trans_items)
t3 = Transaction(3, t3_trans_items)
t4 = Transaction(4, t4_trans_items)
t5 = Transaction(5, t5_trans_items)
database = [t1, t2, t3, t4, t5]

for i in item_list:
    i.twu = calculate_transaction_weight_utility({i}, database)

In [ ]:
a = preparation_procedure(database, item_list, 20, 0.2)

new list: [a, e, b, c, d]


[{a},
 {a, e},
 {a, b, e},
 {a, b, c, e},
 {a, c, e},
 {a, b},
 {a, b, c},
 {a, c},
 {e},
 {b, e},
 {b, c, e},
 {c, e},
 {b},
 {b, c}]